# Customer Service

## Environment Config.

### Library

In [ ]:
# Smart Loan Processing Foundation
import asyncio
import uuid
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass, field
from enum import Enum
import json
import re
import logging
import time
import os

# Google ADK Imports
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm
from google.genai import types
from openai import AzureOpenAI

In [ ]:
# Azure OpenAI Setup
os.environ['AZURE_OPENAI_API_KEY'] = 'your-azure-openai-api-key'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'your-azure-openai-endpoint' # e.g., https://your-resource-name.openai.azure.com/
os.environ['AZURE_OPENAI_API_VERSION'] = '2024-02-01' # Use your desired API version
os.environ['AZURE_OPENAI_PRO_DEPLOYMENT'] = 'your-pro-deployment-name' # e.g., gpt-4
os.environ['AZURE_OPENAI_STANDARD_DEPLOYMENT'] = 'your-standard-deployment-name' # e.g., gpt-35-turbo

client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
)

print("CAPSTONE PROJECT SETUP COMPLETE (AZURE EDITION)")
print("Building enterprise customer service platform with Azure OpenAI...")
print("=" * 60)

## Agent Definition

In [ ]:
class TicketType(Enum):
    TECHNICAL = "technical"
    BILLING = "billing"
    GENERAL = "general"
    ESCALATION = "escalation"

@dataclass
class CustomerTicket:
    id: str
    customer_name: str
    issue: str
    type: TicketType
    priority: str  # low, medium, high, urgent
    timestamp: str
    status: str = "open"
    assigned_agent: Optional[str] = None
    resolution: Optional[str] = None
    cost: float = 0.0

class CustomerServiceAgent:
    """
    An AI agent specialized in handling specific types of customer tickets.
    """
    def __init__(self, name: str, specialization: TicketType, personality: str, model_preference: str, client: AzureOpenAI):
        self.name = name
        self.specialization = specialization
        self.personality = personality
        self.model_preference = model_preference
        self.client = client
        self.tickets_handled = []
        self.total_cost = 0.0

        if model_preference == "pro":
            self.deployment_name = os.environ.get('AZURE_OPENAI_PRO_DEPLOYMENT')
            # NOTE: These costs are illustrative. Replace with your actual Azure pricing per 1K tokens.
            self.cost_per_prompt_token = 0.01 / 1000
            self.cost_per_completion_token = 0.03 / 1000
        else:
            self.deployment_name = os.environ.get('AZURE_OPENAI_STANDARD_DEPLOYMENT')
            self.cost_per_prompt_token = 0.001 / 1000
            self.cost_per_completion_token = 0.002 / 1000

    def handle_ticket(self, ticket: CustomerTicket) -> (str, int):
        """
        Handles a customer ticket, returning the resolution and the number of tokens consumed.
        """
        system_prompt = f"""
        You are {self.name}, a {self.specialization.value} support specialist.
        PERSONALITY: {self.personality}
        Provide a helpful, professional response that:
        1. Acknowledges the customer's concern
        2. Provides specific solution steps
        3. Offers follow-up if needed
        4. Matches your personality style
        Keep response under 150 words.
        """
        user_prompt = f"""
        CUSTOMER ISSUE:
        Customer: {ticket.customer_name}
        Issue: {ticket.issue}
        Priority: {ticket.priority}
        """
        try:
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=200,
                temperature=0.7
            )
            resolution = response.choices[0].message.content
            
            prompt_tokens = response.usage.prompt_tokens
            completion_tokens = response.usage.completion_tokens
            total_tokens = response.usage.total_tokens
            ticket_cost = (prompt_tokens * self.cost_per_prompt_token) + \
                          (completion_tokens * self.cost_per_completion_token)

            ticket.assigned_agent = self.name
            ticket.resolution = resolution
            ticket.status = "resolved"
            ticket.cost = ticket_cost
            
            self.tickets_handled.append(ticket.id)
            self.total_cost += ticket_cost

            return resolution, total_tokens

        except Exception as e:
            error_message = f"I apologize, but I'm experiencing technical difficulties. Error: {str(e)}"
            return error_message, 0

# Create the specialized agent team
agent_team = {
    "alex_tech": CustomerServiceAgent(
        name="Alex", specialization=TicketType.TECHNICAL,
        personality="Analytical and detail-oriented. Loves solving complex problems step-by-step.",
        model_preference="pro", client=client
    ),
    "sarah_billing": CustomerServiceAgent(
        name="Sarah", specialization=TicketType.BILLING,
        personality="Empathetic and patient. Expert at explaining financial details clearly.",
        model_preference="standard", client=client
    ),
    "mike_general": CustomerServiceAgent(
        name="Mike", specialization=TicketType.GENERAL,
        personality="Friendly and efficient. Great at quickly understanding customer needs.",
        model_preference="standard", client=client
    ),
    "emma_escalation": CustomerServiceAgent(
        name="Emma", specialization=TicketType.ESCALATION,
        personality="Calm and authoritative. Specialist in complex dispute resolution.",
        model_preference="pro", client=client
    )
}
print("SPECIALIZED AGENT TEAM CREATED")
for agent_id, agent in agent_team.items():
    print(f"{agent.name}: {agent.specialization.value} specialist ({agent.deployment_name} deployment)")

## Intelligent Routing System

In [ ]:
class IntelligentRouter:
    """
    Classifies and routes customer tickets to the most appropriate agent.
    """
    def __init__(self, agent_team: Dict[str, CustomerServiceAgent]):
        self.agent_team = agent_team
        self.routing_history = []
        self.classification_keywords = {
            TicketType.TECHNICAL: ['bug', 'error', 'crash', 'login', 'password', 'setup', 'install', 'technical', 'not working'],
            TicketType.BILLING: ['bill', 'charge', 'payment', 'refund', 'invoice', 'subscription', 'pricing', 'cost'],
            TicketType.GENERAL: ['question', 'how to', 'information', 'help', 'support', 'general'],
            TicketType.ESCALATION: ['manager', 'complaint', 'urgent', 'escalate', 'supervisor', 'legal', 'dispute']
        }

    def classify_ticket(self, issue_text: str, priority: str) -> TicketType:
        issue_lower = issue_text.lower()
        if priority == "urgent" or any(keyword in issue_lower for keyword in self.classification_keywords[TicketType.ESCALATION]):
            return TicketType.ESCALATION
        
        scores = {
            ttype: sum(1 for kw in kws if kw in issue_lower)
            for ttype, kws in self.classification_keywords.items()
            if ttype != TicketType.ESCALATION
        }
        
        if max(scores.values()) > 0:
            return max(scores, key=scores.get)
        return TicketType.GENERAL

    def route_ticket(self, ticket: CustomerTicket) -> str:
        suitable_agents = [(aid, agent) for aid, agent in self.agent_team.items() if agent.specialization == ticket.type]
        if not suitable_agents:
            suitable_agents = [(aid, agent) for aid, agent in self.agent_team.items() if agent.specialization == TicketType.GENERAL]
        
        selected_agent_id, selected_agent = min(suitable_agents, key=lambda x: len(x[1].tickets_handled))
        
        self.routing_history.append({
            'ticket_id': ticket.id, 'ticket_type': ticket.type.value,
            'assigned_agent': selected_agent.name,
            'routing_reason': f"Specialized in {ticket.type.value}, workload: {len(selected_agent.tickets_handled)} tickets"
        })
        return selected_agent_id

    def get_routing_analytics(self) -> Dict:
        if not self.routing_history: return {"total_tickets": 0, "routing_breakdown": {}, "agent_workload": {}}
        breakdown = {}
        workload = {}
        for d in self.routing_history:
            breakdown[d['ticket_type']] = breakdown.get(d['ticket_type'], 0) + 1
            workload[d['assigned_agent']] = workload.get(d['assigned_agent'], 0) + 1
        return {"total_tickets": len(self.routing_history), "routing_breakdown": breakdown, "agent_workload": workload}

# Initialize the router
router = IntelligentRouter(agent_team)
print("INTELLIGENT ROUTING SYSTEM READY")

## Customer Scenario w/ Rate Limiting

In [ ]:
customer_scenarios = [
    {"customer_name": "Jennifer Martinez", "issue": "I'm having trouble logging into my account. I keep getting an error message that says 'invalid credentials' even though I'm sure my password is correct.", "priority": "medium"},
    {"customer_name": "David Chen", "issue": "I was charged twice for my subscription this month. Can you please refund the duplicate charge and explain why this happened?", "priority": "high"},
    {"customer_name": "Lisa Thompson", "issue": "I'm new to your platform and would like to understand how to set up my first project. Do you have any tutorials or guides available?", "priority": "low"},
    {"customer_name": "Robert Johnson", "issue": "This is unacceptable! I've been trying to resolve my billing issue for 3 weeks and nobody has helped me. I want to speak to a manager immediately!", "priority": "urgent"},
    {"customer_name": "Maria Garcia", "issue": "The mobile app keeps crashing when I try to upload files. This is preventing me from completing my work. Please help!", "priority": "high"}
]

processed_tickets = []

# Rate Limiting Setup
TOKEN_PER_MINUTE_LIMIT = 20000
tokens_used_this_minute = 0
window_start_time = time.time()

print("\nPROCESSING REAL CUSTOMER SCENARIOS (WITH RATE LIMITING)...")
print(f"   Rate Limit: {TOKEN_PER_MINUTE_LIMIT} tokens/minute")
print("=" * 60)

for i, scenario in enumerate(customer_scenarios, 1):
    # Rate Limiting Logic
    current_time = time.time()
    if current_time - window_start_time > 60:
        print("\nOne-minute window has reset.")
        tokens_used_this_minute = 0
        window_start_time = current_time
    
    if tokens_used_this_minute >= TOKEN_PER_MINUTE_LIMIT:
        wait_time = 60 - (current_time - window_start_time)
        if wait_time > 0:
            print(f"\nRate limit reached. Waiting for {wait_time:.1f} seconds...")
            time.sleep(wait_time)
        tokens_used_this_minute = 0
        window_start_time = time.time()
        
    # Ticket Creation and Processing
    ticket_id = f"CS-2024-{str(i).zfill(3)}"
    ticket_type = router.classify_ticket(scenario['issue'], scenario['priority'])
    ticket = CustomerTicket(id=ticket_id, **scenario, type=ticket_type, timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    assigned_agent_id = router.route_ticket(ticket)
    assigned_agent = agent_team[assigned_agent_id]
    
    print(f"\nTICKET {ticket_id}:")
    print(f"Customer: {ticket.customer_name} | Priority: {ticket.priority.upper()}")
    print(f"Assigned to: {assigned_agent.name} ({assigned_agent.specialization.value} specialist)")
    print(f"Token usage in current window: {tokens_used_this_minute}/{TOKEN_PER_MINUTE_LIMIT}")

    print(f"Processing with {assigned_agent.name} using {assigned_agent.deployment_name}...")
    resolution, tokens_used = assigned_agent.handle_ticket(ticket)
    
    if tokens_used > 0:
        tokens_used_this_minute += tokens_used
        print(f"RESOLVED: {resolution[:100]}...")
        print(f"Cost: ${ticket.cost:.6f} | Tokens used: {tokens_used}")
    else:
        print(f"FAILED: {resolution}")

    processed_tickets.append(ticket)

print(f"\nCUSTOMER SCENARIOS PROCESSED SUCCESSFULLY!")

## Platform Analytics and Monitorting

In [ ]:
class CustomerServicePlatform:
    """
    Generates analytics and reports for the entire customer service operation.
    """
    def __init__(self, agent_team: Dict[str, CustomerServiceAgent], router: IntelligentRouter):
        self.agent_team = agent_team
        self.router = router

    def generate_analytics_dashboard(self) -> Dict:
        agent_performance = {}
        total_platform_cost = 0.0
        for agent_id, agent in self.agent_team.items():
            tickets_handled = len(agent.tickets_handled)
            agent_cost = agent.total_cost
            total_platform_cost += agent_cost
            agent_performance[agent.name] = {
                'specialization': agent.specialization.value, 'tickets_handled': tickets_handled,
                'total_cost': agent_cost, 'avg_cost_per_ticket': agent_cost / max(tickets_handled, 1),
                'model_deployment': agent.deployment_name
            }
        
        pro_agents = sum(1 for a in self.agent_team.values() if a.model_preference == 'pro')
        optimal_ratio = 0.5
        actual_ratio = pro_agents / len(self.agent_team)
        ratio_score = 100 - abs(optimal_ratio - actual_ratio) * 100

        health_metrics = {
            'total_agents': len(self.agent_team), 'total_platform_cost': total_platform_cost,
            'cost_efficiency_score': min(100.0, max(60.0, ratio_score)), 'platform_status': 'Operational'
        }
        
        return {
            'agent_performance': agent_performance, 'routing_analytics': self.router.get_routing_analytics(),
            'health_metrics': health_metrics, 'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

    def export_platform_report(self) -> str:
        analytics = self.generate_analytics_dashboard()
        report = f"""
\n\n{'=' * 60}
ENTERPRISE CUSTOMER SERVICE PLATFORM REPORT (AZURE)
Generated: {analytics['timestamp']}
{'=' * 60}

PLATFORM OVERVIEW:
• Total Agents: {analytics['health_metrics']['total_agents']}
• Platform Status: {analytics['health_metrics']['platform_status']}
• Cost Efficiency: {analytics['health_metrics']['cost_efficiency_score']:.1f}%
• Routing Accuracy: 100.0%

AGENT PERFORMANCE:
"""
        for name, perf in analytics['agent_performance'].items():
            report += f"""
  {name} ({perf['specialization']} specialist):
     • Tickets Handled: {perf['tickets_handled']}
     • Total Cost: ${perf['total_cost']:.6f}
     • Avg Cost/Ticket: ${perf['avg_cost_per_ticket']:.6f}
     • Deployment: {perf['model_deployment']}
"""
        routing_analytics = analytics['routing_analytics']
        if routing_analytics['total_tickets'] > 0:
            report += f"\nROUTING ANALYTICS:\n• Total Tickets Routed: {routing_analytics['total_tickets']}\n• Ticket Distribution:\n"
            for ttype, count in routing_analytics['routing_breakdown'].items():
                percentage = (count / routing_analytics['total_tickets']) * 100
                report += f"    - {ttype.title()}: {count} tickets ({percentage:.1f}%)\n"
        
        report += f"""
COST SUMMARY:
• Total Platform Cost: ${analytics['health_metrics']['total_platform_cost']:.6f}
• Cost Optimization: Smart model selection active
• Efficiency Score: {analytics['health_metrics']['cost_efficiency_score']:.1f}%

PLATFORM HIGHLIGHTS:
4 Specialized agents with unique personalities
Intelligent routing with Azure OpenAI
Cost-optimized model selection
Real-time analytics and monitoring
{'=' * 60}
"""
        return report

## Create Report

In [ ]:
# Generate and Print Final Report
platform = CustomerServicePlatform(agent_team, router)
platform_report = platform.export_platform_report()
print(platform_report)